# Hello

![](./localhost.png)

In [12]:
# Import py2neo and connect to Neo4j
from py2neo import Graph

# Import pandas to make reading the outputs easier
import pandas as pd

# Import number generator to generate a random user ID and product ID to query
from random import randint

# textblob for the textblob-NLP stuff
from textblob import TextBlob

# just an example, replace with credentials for your own Neo4j instance
graph = Graph(bolt=True, host="localhost", http_port=7687, user='neo4j', password='kiss')

In [ ]:
# Hello world, sanity check
graph.run("MATCH (a) RETURN COUNT(a) AS numberOfNodes").evaluate()

In [ ]:
# # Create constraints
# graph.run("CREATE CONSTRAINT ON (c:Category) ASSERT c.name IS UNIQUE;")
# graph.run("CREATE CONSTRAINT ON (r:Recipe) ASSERT r.name is UNIQUE;")

In [ ]:
# # Using APOC for the JSON import

# graph.run("""
# CALL apoc.load.json("file:///epi_recipe_json_cleaned.json")
# YIELD value AS line
# CREATE (recipe:Recipe {name: line.title, rating: toInteger(line.rating), ingredients: line.ingredients, directions: line.directions})
# WITH recipe, line.categories AS categories
# UNWIND categories AS cat
# MERGE (c:Category {name:cat})
# CREATE (recipe)-[:TAGGED_AS]->(c)""")

## Content-based Filtering

Simple version, using just the number of common categories

In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

In [19]:
pd.DataFrame(graph.data("""
MATCH (m:Recipe)-[:TAGGED_AS]->(g:Category)<-[:TAGGED_AS]-(rec:Recipe)
WHERE m.name = 'Potato and Fennel Soup Hodge'
WITH rec, COLLECT(g.name) AS categories, COUNT(*) AS commonCategories
RETURN rec.name, categories, commonCategories
ORDER BY commonCategories DESC 
LIMIT 10"""))


,categories,commonCategories,rec.name
0,"[Gourmet, Soup/Stew, Fennel, Vegetable, Dairy]",5,"Carrot, Fennel, and Orange Soup"
1,"[Soup/Stew, Fennel, Vegetable, Potato, Dairy]",5,Fresh Fennel Vichyssoise
2,"[Gourmet, Soup/Stew, Vegetable, Potato, Dairy]",5,Salmon Chowder
3,"[Soup/Stew, Fennel, Vegetable, Potato]",4,"Potato, Leek and Fennel Soup"
4,"[Gourmet, Soup/Stew, Fennel, Potato]",4,"Lobster ""Potpies"""
5,"[Gourmet, Soup/Stew, Vegetable, Dairy]",4,Zucchini Ginger Cupcakes
6,"[Soup/Stew, Vegetable, Potato, Dairy]",4,Cream of Potato Soup with Watercress
7,"[Gourmet, Soup/Stew, Vegetable, Dairy]",4,Sweet Red Pepper Soup
8,"[Gourmet, Soup/Stew, Vegetable, Dairy]",4,Lobster Bisque
9,"[Gourmet, Soup/Stew, Vegetable, Dairy]",4,Chilled Minted Zucchini Soup


In [21]:
pd.DataFrame(graph.data("""
MATCH (r:Recipe {name: "Spicy Noodle Soup"})-[:TAGGED_AS]->(c:Category)<-[:TAGGED_AS]-(other:Recipe)
WITH r, other, COUNT(c) AS intersection, COLLECT(c.name) AS i

MATCH (r)-[:TAGGED_AS]->(rc:Category)
WITH r, other, intersection, i, COLLECT(rc.name) AS s1

MATCH (other)-[:TAGGED_AS]->(oc:Category)
WITH r, other, intersection, i,  s1, COLLECT(oc.name) AS s2

WITH r, other, intersection, s1, s2

WITH r, other, intersection, s1+filter(x IN s2 WHERE NOT x IN s1) AS union, s1, s2

RETURN other.name AS recommendation, s1, s2,((1.0*intersection)/SIZE(union)) AS jaccard 
ORDER BY jaccard DESC 
LIMIT 10"""))


,jaccard,recommendation,s1,s2
0,0.357143,Thai Chicken Stock,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Soup/Stew, Gourmet, Garlic, Chicken, Cilantro, Ginger]"
1,0.352941,Curried Chicken Coconut Noodle Soup,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Chicken, Ginger, Soup/Stew, Curry, Winter, Pasta, Coconut, Gourmet, Lemongrass, Cilantro]"
2,0.333333,Hainanese Chicken Rice,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Gourmet, Hot Pepper, Rice, Ginger, Garlic, Chicken, Soup/Stew]"
3,0.300000,Aromatic Shrimp And Noodle Medicine Soup,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Lemongrass, Soup/Stew, Bon Appétit, Dairy Free, Winter, Healthy, Dinner, Ginger, Green Onion/Scallion, Noodle, Shrimp, Peanut Free, Soy Free]"
4,0.294118,Coconut Sauce,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Coconut, Quick & Easy, Gourmet, Lemongrass, Chicken, Blender, Herb, Ginger, Sauce]"
5,0.294118,"Chicken, Red Pepper, and Green Bean Stir-Fry","[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Ginger, Chicken, Bon Appétit, Sesame, Lemongrass, Spring, Quick & Easy, Stir-Fry, Pepper]"
6,0.294118,Matzo Balls in Southeast Asian Broth,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Cilantro, Bon Appétit, Kosher, Lemongrass, Passover, Spring, Chicken, Ginger, Soup/Stew]"
7,0.291667,Thai Shrimp Soup with Lemon and Jalapenos,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Chicken, Soup/Stew, Pork, Ginger, Lunch, Shellfish, Coconut, Lemon, Healthy, Shrimp, No Sugar Added, Soy Free, Tree Nut Free, Peanut Free, Dairy Free, Sugar Conscious, Bon Appétit, Jalapeño]"
8,0.285714,Coconut Shrimp with Tamarind Ginger Sauce,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Peanut Free, Tree Nut Free, Soy Free, Tamarind, Gourmet, Pescatarian, Dairy Free, Cocktail Party, Coconut, Shrimp, Summer, Ginger, Appetizer, Fry]"
9,0.285714,Chile-Pepper Water,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Ginger, Pepper, Winter, Gourmet, Garlic]"


Add a rating comparison, i.e. the rating of the recommendation can't be lower than the rating of the recipe being searched.

In [22]:
pd.DataFrame(graph.data("""
MATCH (r:Recipe {name: "Spicy Noodle Soup"})-[:TAGGED_AS]->(c:Category)<-[:TAGGED_AS]-(other:Recipe)
WITH r, other, COUNT(c) AS intersection, COLLECT(c.name) AS i

MATCH (r)-[:TAGGED_AS]->(rc:Category)
WITH r, other, intersection, i, COLLECT(rc.name) AS s1

MATCH (other)-[:TAGGED_AS]->(oc:Category)
WITH r, other, intersection, i,  s1, COLLECT(oc.name) AS s2

WITH r, other, intersection, s1, s2

WITH r, other, intersection, s1+filter(x IN s2 WHERE NOT x IN s1) AS union, s1, s2

WHERE other.rating >= r.rating

RETURN other.name AS recommendation, s1,s2,((1.0*intersection)/SIZE(union)) AS jaccard 
ORDER BY jaccard DESC 
LIMIT 10"""))


,jaccard,recommendation,s1,s2
0,0.333333,Hainanese Chicken Rice,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Gourmet, Hot Pepper, Rice, Ginger, Garlic, Chicken, Soup/Stew]"
1,0.300000,Aromatic Shrimp And Noodle Medicine Soup,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Lemongrass, Soup/Stew, Bon Appétit, Dairy Free, Winter, Healthy, Dinner, Ginger, Green Onion/Scallion, Noodle, Shrimp, Peanut Free, Soy Free]"
2,0.294118,Coconut Sauce,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Coconut, Quick & Easy, Gourmet, Lemongrass, Chicken, Blender, Herb, Ginger, Sauce]"
3,0.291667,Thai Shrimp Soup with Lemon and Jalapenos,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Chicken, Soup/Stew, Pork, Ginger, Lunch, Shellfish, Coconut, Lemon, Healthy, Shrimp, No Sugar Added, Soy Free, Tree Nut Free, Peanut Free, Dairy Free, Sugar Conscious, Bon Appétit, Jalapeño]"
4,0.285714,Coconut Shrimp with Tamarind Ginger Sauce,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Peanut Free, Tree Nut Free, Soy Free, Tamarind, Gourmet, Pescatarian, Dairy Free, Cocktail Party, Coconut, Shrimp, Summer, Ginger, Appetizer, Fry]"
5,0.280000,Pigeon-Pea Soup,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[No Sugar Added, Dairy Free, Peanut Free, Tree Nut Free, Soy Free, Plantain, Jalapeño, Cilantro, Gourmet, Sausage, Hot Pepper, Butternut Squash, Spring, Pork, Pepper, Lunch, Vegetable, Garlic, Soup/Stew]"
6,0.277778,Thai-Style Chicken and Rice Soup,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Coconut, Rice, Poultry, Ginger, Chicken, Soup/Stew, Winter, Curry, Gourmet, Cilantro]"
7,0.277778,Chicken and Cashew Stir-Fry,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Wok, Chicken, Quick & Easy, Gourmet, Pepper, Stir-Fry, Nut, Onion, Garlic, Ginger]"
8,0.272727,Shrimp Sambal,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Macadamia Nut, Shrimp, Spice, Hot Pepper, Spring, Simmer, Gourmet, Food Processor, Garlic, Ginger, Nut, Onion, Sauté, Quick & Easy, Coconut]"
9,0.266667,Chicken with Garlic Sauce,"[Soup/Stew, Coconut, Breakfast, Spring, Shrimp, Garlic, Chicken, Pepper, Ginger, Gourmet, Lemongrass, Peanut Free, Dairy Free]","[Garlic, Herb, Chicken, Gourmet, Poultry, Spring]"


In [23]:
product_name = "Organic Large Extra Fancy Fuji Apple"
words = product_name.split()

In [25]:
words

['Organic', 'Large', 'Extra', 'Fancy', 'Fuji', 'Apple']

In [27]:
pd.DataFrame(graph.data("""
MATCH (r:Recipe)-[:TAGGED_AS]->(c:Category)<-[:TAGGED_AS]-(other:Recipe)
WHERE c.name IN ['Organic', 'Large', 'Extra', 'Fancy', 'Fuji', 'Apple']
WITH r, other, COUNT(c) AS intersection, COLLECT(c.name) AS i

MATCH (r)-[:TAGGED_AS]->(rc:Category)
WITH r, other, intersection, i, COLLECT(rc.name) AS s1

MATCH (other)-[:TAGGED_AS]->(oc:Category)
WITH r, other, intersection, i,  s1, COLLECT(oc.name) AS s2

WITH r, other, intersection, s1, s2

WITH r, other, intersection, s1+filter(x IN s2 WHERE NOT x IN s1) AS union, s1, s2

RETURN other.name AS recommendation, s1, s2,((1.0*intersection)/SIZE(union)) AS jaccard 
ORDER BY jaccard DESC 
LIMIT 10"""))

,jaccard,recommendation,s1,s2
0,0.166667,Apple Cobblers,"[Gourmet, Apple, Bake, Side, Fruit]","[Apple, Fall, Fruit, Bake, Gourmet]"
1,0.166667,Cumin Apple Chips,"[Apple, Fall, Fruit, Bake, Gourmet]","[Gourmet, Apple, Bake, Side, Fruit]"
2,0.166667,Apple Cobblers,"[Fall, Gourmet, Apple, Bake, Dessert, Fruit]","[Apple, Fall, Fruit, Bake, Gourmet]"
3,0.166667,Glazed Apple Rings,"[Apple, Fall, Fruit, Bake, Gourmet]","[Fall, Gourmet, Apple, Bake, Dessert, Fruit]"
4,0.142857,Calvados Tarte Tatin,"[Fall, Gourmet, Apple, Bake, Dessert, Fruit]","[Dessert, Liqueur, Gourmet, Fall, Apple, Bake]"
5,0.142857,Apple Cobblers,"[Phyllo/Puff Pastry Dough, Gourmet, Apple, Fall, Dessert, Bake]","[Apple, Fall, Fruit, Bake, Gourmet]"
6,0.142857,Thin Apple Tarts,"[Dessert, Liqueur, Gourmet, Fall, Apple, Bake]","[Phyllo/Puff Pastry Dough, Gourmet, Apple, Fall, Dessert, Bake]"
7,0.142857,Cumin Apple Chips,"[Fall, Gourmet, Apple, Bake, Dessert, Fruit]","[Gourmet, Apple, Bake, Side, Fruit]"
8,0.142857,Calvados Tarte Tatin,"[Phyllo/Puff Pastry Dough, Gourmet, Apple, Fall, Dessert, Bake]","[Dessert, Liqueur, Gourmet, Fall, Apple, Bake]"
9,0.142857,Glazed Apple Rings,"[Phyllo/Puff Pastry Dough, Gourmet, Apple, Fall, Dessert, Bake]","[Fall, Gourmet, Apple, Bake, Dessert, Fruit]"


But the above query took a long while to run. This might be the more computationaly efficient way...

In [36]:
pd.DataFrame(graph.data("""
MATCH (m:Recipe)-[:TAGGED_AS]->(c:Category)<-[:TAGGED_AS]-(rec:Recipe)
WHERE c.name IN ['Organic', 'Large', 'Extra', 'Fancy', 'Fuji', 'Apple']
WITH rec, COLLECT(DISTINCT c.name) AS categories, COUNT(DISTINCT c) AS commonCategories
RETURN rec.name, categories, commonCategories
ORDER BY commonCategories DESC 
LIMIT 10"""))

,categories,commonCategories,rec.name
0,[Apple],1,Apple and Caraway Tartlets with Cinnamon-Clove Ice Cream and Cider-Caramel Sauce
1,[Apple],1,Apple Cheddar Bread Pudding
2,[Apple],1,Kielbasa with Golden Onions and Apple
3,[Apple],1,Calf's Liver with Apples and Onions
4,[Apple],1,Apple Pancakes with Cinnamon Butter
5,[Apple],1,Cranberry Apricot Pecan Strudel
6,[Apple],1,Ben's Chunky Applesauce
7,[Apple],1,Mincemeat Muffins
8,[Apple],1,Warm Baked Apples with Cranberry-Caramel Sauce
9,[Apple],1,Warm Sweet-Potato Pudding with Apples and Chestnuts
